In [7]:
# !pip install sqlalchemy


In [18]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [9]:
# !ln -s ../../stockdata .
# !ln -s ../../data .

In [19]:
# engine = create_engine('sqlite:///NQ_DB.db')
engine = create_engine('sqlite:///data/ESNQ_DB.db')

In [20]:

table_names = inspect(engine).get_table_names()

print("Tables in the database:")
table_names

Tables in the database:


['ES1M', 'ES30M', 'ES5M', 'NQ1M', 'NQ30M', 'NQ5M']

In [12]:
files = !ls stockdata/ES*30M.csv
for file in files:
    print(file)


stockdata/ES_2024-01-01_2024-06-10_30M.csv
stockdata/ES_2024-01-01_2024-06-14_30M.csv


In [13]:
def sql_importer (df, table_name):
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        df.to_sql(table_name, engine, if_exists='append')
        print (f'New table created for {table_name} with {str(len(df))} rows')
    else:
        max_date = pd.read_sql(f'SELECT MAX(datetime) FROM {table_name}', engine).values[0][0]
        print(max_date)
        df = df[df.datetime > max_date]
        df.to_sql(table_name, engine, if_exists='append')
        print(str(len(df)) + ' new rows imported to DB')

In [14]:
for file in files:
    df = pd.read_csv(file)
    sql_importer(df, 'ES30M')

2024-06-14 15:30:00
0 new rows imported to DB
2024-06-14 15:30:00
0 new rows imported to DB


In [15]:
pd.read_sql('ES30M', engine)

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-01-01 17:00:00,CME_MINI:ES1!,4818.00,4822.50,4815.75,4821.75,8431.0
1,1,2024-01-01 17:30:00,CME_MINI:ES1!,4822.00,4822.25,4820.25,4821.00,4871.0
2,2,2024-01-01 18:00:00,CME_MINI:ES1!,4820.75,4821.50,4819.00,4819.25,2533.0
3,3,2024-01-01 18:30:00,CME_MINI:ES1!,4819.25,4820.50,4818.75,4819.75,2075.0
4,4,2024-01-01 19:00:00,CME_MINI:ES1!,4819.75,4822.75,4819.75,4821.25,8181.0
...,...,...,...,...,...,...,...,...
5399,5399,2024-06-14 13:30:00,CME_MINI:ES1!,5493.50,5498.00,5488.00,5495.50,19713.0
5400,5400,2024-06-14 14:00:00,CME_MINI:ES1!,5495.25,5499.00,5493.50,5498.00,16855.0
5401,5401,2024-06-14 14:30:00,CME_MINI:ES1!,5498.00,5503.25,5493.25,5502.25,35509.0
5402,5402,2024-06-14 15:00:00,CME_MINI:ES1!,5502.50,5502.75,5498.25,5499.75,12016.0


In [16]:
df = pd.read_sql('SELECT * FROM NQ30M', con=engine)
df

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-01-01 17:00:00,CME_MINI:NQ1!,17019.00,17037.00,17013.75,17033.25,3466.0
1,1,2024-01-01 17:30:00,CME_MINI:NQ1!,17033.25,17036.25,17027.25,17029.50,1116.0
2,2,2024-01-01 18:00:00,CME_MINI:NQ1!,17030.00,17030.75,17018.50,17019.75,1222.0
3,3,2024-01-01 18:30:00,CME_MINI:NQ1!,17019.75,17028.00,17018.50,17027.00,1048.0
4,4,2024-01-01 19:00:00,CME_MINI:NQ1!,17026.75,17038.50,17026.25,17029.25,2664.0
...,...,...,...,...,...,...,...,...
5399,5399,2024-06-14 13:30:00,CME_MINI:NQ1!,19917.50,19935.50,19892.75,19926.00,7676.0
5400,5400,2024-06-14 14:00:00,CME_MINI:NQ1!,19925.50,19938.50,19917.75,19932.00,6800.0
5401,5401,2024-06-14 14:30:00,CME_MINI:NQ1!,19932.00,19951.50,19906.00,19948.00,12757.0
5402,5402,2024-06-14 15:00:00,CME_MINI:NQ1!,19948.25,19959.00,19940.00,19956.50,4394.0


In [17]:
df = pd.read_sql('SELECT datetime, open, close FROM NQ5M', con=engine)
df

,datetime,open,close
0,2024-05-12 17:00:00,18238.00,18241.75
1,2024-05-12 17:05:00,18241.75,18230.75
2,2024-05-12 17:10:00,18230.75,18234.00
3,2024-05-12 17:15:00,18233.75,18239.00
4,2024-05-12 17:20:00,18238.50,18241.00
...,...,...,...
21886,2024-08-30 15:35:00,19604.75,19602.00
21887,2024-08-30 15:40:00,19601.75,19599.75
21888,2024-08-30 15:45:00,19599.75,19596.00
21889,2024-08-30 15:50:00,19596.50,19596.00


In [19]:
# df.describe().style.background_gradient(cmap='coolwarm')
df.describe()
df.describe().style.background_gradient(cmap='coolwarm')

,open,close
count,21891.000000,21891.000000
mean,19433.779898,19433.831026
std,725.540002,725.475703
min,17426.000000,17425.250000
25%,18778.750000,18779.000000
50%,19509.000000,19509.250000
75%,19974.250000,19974.375000
max,20972.250000,20971.750000


In [20]:
import pandas_ta as ta
macd = df.ta.macd(fast=13, slow=26)
macd

,MACD_13_26_9,MACDh_13_26_9,MACDs_13_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
21886,26.866794,0.975748,25.891046
21887,25.752240,-0.111044,25.863285
21888,24.337227,-1.220846,25.558073
21889,22.956664,-2.081128,25.037791


In [21]:
rsi = df.ta.rsi()
rsi

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
21886    67.754123
21887    66.532842
21888    64.447768
21889    64.447768
21890    63.524232
Name: RSI_14, Length: 21891, dtype: float64

In [23]:
from sqlalchemy import create_engine, MetaData

# Setup your engine
# engine = create_engine('your_database_url')
# engine = create_engine('sqlite:///TEST_DB.db')
engine = create_engine('sqlite:///data/ESNQ_DB.db')

# Reflect the database schema to get all table names
metadata = MetaData()
metadata.reflect(bind=engine)

print("Tables in the database:")
for table_name in metadata.tables.keys():
    print(table_name)

Tables in the database:
ES1M
ES30M
ES5M
NQ1M
NQ30M
NQ5M
